# Classificação de Renda com Dataset Adult (UCI)
Este notebook demonstra a classificação binária de renda (acima ou abaixo de 50K/ano) usando o dataset Adult do UCI Machine Learning Repository e um modelo sequencial simples em Keras.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Carregar os dados (assumindo que os arquivos CSV estão no mesmo diretório ou caminho especificado)
# Para rodar no Google Colab, você pode precisar fazer upload dos arquivos originais (adult.data e adult.test) ou dos arquivos processados (adult_processed_train.csv e adult_processed_test.csv).
# Se for usar os arquivos processados, descomente as linhas abaixo e comente as linhas de pré-processamento:
# df_train = pd.read_csv('adult_processed_train.csv')
# df_test = pd.read_csv('adult_processed_test.csv')

column_names = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num',
    'marital-status', 'occupation', 'relationship', 'race', 'sex',
    'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'
]

# Ler os arquivos inferindo o cabeçalho e especificando a codificação correta
df_train = pd.read_csv('adult_processed_train.csv', encoding='latin-1')
df_test = pd.read_excel('adult_processed_test.xlsx')
# Remover linhas com valores ausentes
df_train.dropna(inplace=True)
df_test.dropna(inplace=True)

# Pré-processamento da variável target
df_train['income'] = df_train['income'].apply(lambda x: 1 if x == '>50K' else 0)
df_test['income'] = df_test['income'].apply(lambda x: 1 if x == '>50K.' else 0)

# Separar features (X) e target (y)
X_train = df_train.drop('income', axis=1)
y_train = df_train['income']
X_test = df_test.drop('income', axis=1)
y_test = df_test['income']

# Identificar colunas categóricas e numéricas
categorical_cols = X_train.select_dtypes(include=['object']).columns
numerical_cols = X_train.select_dtypes(include=np.number).columns

# One-Hot Encoding para colunas categóricas
X_train = pd.get_dummies(X_train, columns=categorical_cols, drop_first=True)
X_test = pd.get_dummies(X_test, columns=categorical_cols, drop_first=True)

# Alinhar colunas após One-Hot Encoding
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)
X_test = X_test.loc[:, X_train.columns]

# Escalonamento das features numéricas
scaler = StandardScaler()
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

print(f'Shape de X_train: {X_train.shape}')
print(f'Shape de X_test: {X_test.shape}')
print(f'Shape de y_train: {y_train.shape}')
print(f'Shape de y_test: {y_test.shape}')

## Exploração do Dataset
O dataset Adult contém 48842 instâncias e 14 features, com a tarefa de prever se a renda de um indivíduo excede 50K/ano. Após a remoção de valores ausentes e o pré-processamento (One-Hot Encoding e escalonamento), o número de features pode aumentar devido às variáveis categóricas.

**Características Principais:**
*   **Número de Amostras (após limpeza):** Aproximadamente 45.222 (32.561 treino, 16.281 teste no original, mas após remoção de NaNs e alinhamento, os números podem variar ligeiramente).
*   **Número de Features:** 14 features originais (numéricas e categóricas), expandidas após One-Hot Encoding.
*   **Tarefa de Classificação:** Binária (renda >50K ou <=50K).
*   **Features:** Idade, classe de trabalho, peso final (fnlwgt), educação, número de educação, estado civil, ocupação, relacionamento, raça, sexo, ganho de capital, perda de capital, horas por semana, país de origem.

In [ ]:
# Construir o modelo Keras
model = keras.Sequential([
    layers.Dense(1, activation='sigmoid', input_shape=(X_train.shape[1],))
])

# Compilar o modelo
# Otimizador Adam: Um algoritmo de otimização adaptativo que ajusta as taxas de aprendizado para cada parâmetro individualmente.
# Função de Perda binary_crossentropy: Usada para problemas de classificação binária, mede a diferença entre a distribuição de probabilidade prevista e a verdadeira.
# Métrica accuracy: Proporção de previsões corretas.
# Métrica F1: Média harmônica da precisão e recall, útil para datasets desbalanceados.
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.F1Score(average='weighted')]
)

model.summary()

## Treinamento do Modelo
O modelo será treinado por 50 épocas com um batch size de 10. Uma parte do conjunto de treino será usada para validação.

In [ ]:
# Treinar o modelo
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=10,
    verbose=1,
    validation_split=0.2
)

## Avaliação e Interpretação dos Resultados
Após o treinamento, o modelo será avaliado no conjunto de teste para calcular a acurácia e o F1-Score. Esses resultados nos darão uma indicação do desempenho do modelo na tarefa de classificação de renda.

In [ ]:
# Prever no conjunto de teste
y_pred_proba = model.predict(X_test)
y_pred = (y_pred_proba > 0.5).astype(int)

# Calcular acurácia e F1-Score
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Acurácia no conjunto de teste: {accuracy:.4f}')
print(f'F1-Score no conjunto de teste: {f1:.4f}')

# Interpretação dos resultados
print('Interpretação:')
print('A acurácia indica a proporção de previsões corretas. O F1-Score é particularmente útil em datasets desbalanceados, pois considera tanto a precisão quanto o recall.')
print('Um modelo com uma única camada Dense pode ter um desempenho limitado para um dataset complexo como o Adult. Possíveis melhorias incluem:')
print('- Adicionar mais camadas ocultas.')
print('- Aumentar o número de neurônios nas camadas.')
print('- Experimentar diferentes funções de ativação.')
print('- Ajustar hiperparâmetros como taxa de aprendizado, batch size e número de épocas.')
print('- Realizar engenharia de features mais avançada.')
print('- Utilizar técnicas de regularização para evitar overfitting.')

NameError: name 'model' is not defined